In [ ]:
import os, sys
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../utils')
import pandas as pd
import numpy as np
from queryHelper import prodFetch, adbFetch
from databaseHelper import *
from sheetHelper import *
from datetime import datetime as dt, timedelta
import psycopg2
import pygsheets as pg
import requests
import fitz
import logging
from io import BytesIO
from google.oauth2 import service_account
from google.auth.transport.requests import Request
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

In [ ]:
yesterday_date = pd.Timestamp.today().date() - timedelta(days=1)
last_week_date = pd.Timestamp.today().date() - timedelta(days=7)

print("Yesterday's date: ", yesterday_date)
print("Last week's date: ", last_week_date)

In [ ]:
dfDrivers = adbFetch(f"""select driverId, date, zoneId from dailyDriversHistories where driverId like 'D%' and clientId = 'BS00' and vehicleType != 'E-2w' and status in ('active', 'inactive') and deletedAt is null and isBaasDriver != 1 and date in ('{yesterday_date}', '{last_week_date}')""")

dfDrivers

In [ ]:
dfDrivers_yesterday = dfDrivers[dfDrivers['date'] == yesterday_date].copy()
dfDrivers_lastweek = dfDrivers[dfDrivers['date'] == last_week_date].copy()

print("Yesterday's Drivers:")
print(dfDrivers_yesterday.head())

print("\nLast Week's Drivers:")
print(dfDrivers_lastweek.head())

In [ ]:
txns = prodFetch(f""" select driverId, date, sum(netGMV)/100 as revenue, sum(swaps) swaps, 1 as opsFlag from transactions where date in ('{yesterday_date}', '{last_week_date}') and deletedAt is null group by driverId, date""")

txns

In [ ]:
txns_yesterday = txns[txns['date'] == yesterday_date].copy()
txns_lastweek = txns[txns['date'] == last_week_date].copy()

print("Yesterday's Transactions:")
print(txns_yesterday.head())
print("\nLast Week's Transactions:")
print(txns_lastweek.head())

In [ ]:
dfdsh = prodFetch(f""" select driverId, date, status from driverStatusHistories where date in ('{yesterday_date}', '{last_week_date}') and deletedAt is null and status = 'active' """)

dfdsh

In [ ]:
dfdsh_yesterday = dfdsh[dfdsh['date'] == yesterday_date].copy()
dfdsh_lastweek = dfdsh[dfdsh['date'] == last_week_date].copy()

print("Yesterday's Transactions:")
print(dfdsh_yesterday.head())
print("\nLast Week's Transactions:")
print(dfdsh_lastweek.head())

In [ ]:
conn = psycopg2.connect(
    dbname="operations_manager_prod",
    user="sarthak_sachdev",
    password="Sarthak@12345",
    host="operation.replica.upgrid.in",
    port="5432"
)

print("PostgreSQL Connection Established")

battery_query = f"""
    SELECT complainant_id AS driverId,
           (created_at + INTERVAL '330 minutes')::DATE AS battery_ticket_date,
           COUNT(*) AS battery_totalTickets
    FROM tickets
    WHERE (created_at + INTERVAL '330 minutes')::DATE IN ('{yesterday_date}')
      AND deleted_at IS NULL
      AND complainant_id LIKE 'D%'
      AND issue_id IN (401, 922, 923, 3201, 2862, 984)
    GROUP BY complainant_id, battery_ticket_date
"""

wait_query = f"""
    SELECT complainant_id AS driverId,
           (created_at + INTERVAL '330 minutes')::DATE AS wait_ticket_date,
           COUNT(*) AS wait_totalTickets
    FROM tickets
    WHERE (created_at + INTERVAL '330 minutes')::DATE IN ('{yesterday_date}')
      AND deleted_at IS NULL
      AND complainant_id LIKE 'D%'
      AND issue_id IN (399, 380, 576, 603, 583, 626, 639, 662, 621, 620,
                       636, 1916, 1952, 2427, 2527, 2526, 3146, 793, 3166)
    GROUP BY complainant_id, wait_ticket_date
"""

meter_query = f"""
    SELECT complainant_id AS driverId,
           (created_at + INTERVAL '330 minutes')::DATE AS meter_ticket_date,
           COUNT(*) AS meter_totalTickets
    FROM tickets
    WHERE (created_at + INTERVAL '330 minutes')::DATE IN ('{yesterday_date}')
      AND deleted_at IS NULL
      AND complainant_id LIKE 'D%'
      AND issue_id IN (2488, 2672, 2676, 2773, 2846, 2947, 2983, 2990, 3254, 3255)
    GROUP BY complainant_id, meter_ticket_date
"""

dfBatteryTickets = pd.read_sql(battery_query, conn)
dfWaitTickets = pd.read_sql(wait_query, conn)
dfMeterTickets = pd.read_sql(meter_query, conn)

conn.close()

print(dfBatteryTickets.head())
print(dfWaitTickets.head())
print(dfMeterTickets.head())


In [ ]:
dfCrossSwapTickets = adbFetch(f"""select date, complainant_id driverId, count(*) as crossSwap_totaltickets from ticketModelNew where category_issue in ('Fraud Report - Fake Swap Created by Partner') and date = '{yesterday_date}' and complainant_type in ('driver') group by driverId, date""")

dfCrossSwapTickets

In [ ]:
# dfNonOps = prodFetch(f"""select driverId, date from driverLeaves where driverId like 'D%' and deletedAt is null and date between '{last_week_date}' and '{yesterday_date}' """)

# dfNonOps['date'] = pd.to_datetime(dfNonOps['date'])

# driver_leave_days = dfNonOps.groupby('driverId')['date'].nunique().reset_index(name='leave_days_last_7')

# driver_leave_days['RT_case'] = (driver_leave_days['leave_days_last_7'] >= 7).astype(int)

# rt_final = driver_leave_days[driver_leave_days['RT_case'] == 1].copy()

# rt_final
rt = read("https://docs.google.com/spreadsheets/d/19pnhX57q6pZxdfG6DhBBXI7eBQ6U52Y-eoBBcM7pfWk/edit?gid=0#gid=0", "Tasks")

rt

In [ ]:
rt['Case Date'] = pd.to_datetime(rt['Case Date']).dt.date

rt = rt[rt['Case Date'] == yesterday_date].copy()
rt

In [ ]:
rt_final = rt.groupby('zoneId')['Driver'].nunique().reset_index(name='rt_cases')

rt_final

In [ ]:
final = dfDrivers_yesterday.merge(txns_yesterday, on=['driverId', 'date'], how='left')

final.fillna(0, inplace=True)
final

In [ ]:
final2 = dfDrivers_lastweek.merge(txns_lastweek, on=['driverId', 'date'], how='left')

final2.fillna(0, inplace=True)
final2

In [ ]:
dfdsh_lastweek

In [ ]:
finalLastWeek = final2.merge(dfdsh_lastweek, on=['driverId', 'date'], how='left')

finalLastWeek.fillna("inactive", inplace=True)
finalLastWeek

In [ ]:
final_merged = final.merge(dfdsh, on=['driverId', 'date'], how='left')
final_merged.fillna("inactive", inplace=True)
final_merged

In [ ]:
final_merged = final_merged.merge(dfBatteryTickets, left_on=['driverId', 'date'], right_on=['driverid', 'battery_ticket_date'], how='left').drop(columns=['battery_ticket_date', 'driverid'])

final_merged.fillna("", inplace=True)
final_merged

In [ ]:
final_merged = final_merged.merge(dfWaitTickets, left_on=['driverId', 'date'], right_on=['driverid', 'wait_ticket_date'], how='left').drop(columns=['wait_ticket_date', 'driverid'])

final_merged.fillna("", inplace=True)
final_merged

In [ ]:
final_merged = final_merged.merge(dfMeterTickets, left_on=['driverId', 'date'], right_on=['driverid', 'meter_ticket_date'], how='left').drop(columns=['meter_ticket_date', 'driverid'])

final_merged.fillna("", inplace=True)
final_merged

In [ ]:
final_merged = final_merged.merge(dfCrossSwapTickets, on=['driverId', 'date'], how='left')

final_merged.fillna(0, inplace=True)
final_merged

In [ ]:
dfInActivity = prodFetch(f"""SELECT ab.inactiveDate date, driverId, ab.zoneId, count(*) inactivity
FROM
(
with inactivityData as(
select
date(date_add(createdAt, interval 330 minute)) inactiveDate,
driverId,
changedBy,
CASE
WHEN changedBy in (SELECT distinct skId from dsk) THEN 'JIA'
WHEN changedBy IS NULL THEN NULL
ELSE 'OSC'
END inactive_APPusage,
statusChangedVia,
batteryDepositedTo,
CASE
WHEN batteryDepositedTo like 'DSK%' THEN REPLACE(batteryDepositedTo, "DSK", "P")
ELSE batteryDepositedTo
END batteryDepositedToR
from driverAudits
where deletedAt IS NULL
AND oldStatus IN ('absent', 'active')
AND newStatus='onLeave'
AND date(date_add(createdAt, interval 330 minute)) = '{yesterday_date}'
-- AND status='verified'
),
p1 as (
select
i.*,
dbas.partnerId,
CASE WHEN isActiveDsk=1 THEN 'DSK' ELSE 'NON_DSK' END dsk_Status,
isActiveDsk,
dskId, dskCreatedAt, dskUpdatedAt,
CASE
WHEN isActiveDsk=1 THEN 'DSK'
WHEN isActiveDsk=0 AND dskId IS NOT NULL AND dsk.status='inactive' AND
(inactiveDate BETWEEN dskCreatedAt AND dskUpdatedAt) THEN 'DSK'
WHEN isActiveDsk=0 AND dskId IS NOT NULL AND dskCreatedAt<=inactiveDate AND dsk.status='active' THEN 'DSK'
ELSE 'NON_DSK'
END finalDskStatus,
dsk.status
from inactivityData i LEFT JOIN
(select date, partnerId, isActiveDsk from dailyBatteriesAtStations where deletedAt IS NULL) dbas
ON i.batteryDepositedToR=dbas.partnerId AND i.inactiveDate=dbas.date
LEFT JOIN
(select
id dskId, partnerId, status, zoneId,
date(date_add(createdAt, interval 330 minute)) dskCreatedAt,
date(date_add(updatedAt, interval 330 minute)) dskUpdatedAt
from dsk where deletedAt IS NULL
-- AND status='active'
) dsk ON i.batteryDepositedToR=dsk.partnerId
)
select
distinct p1.inactiveDate, driverId, p.zoneId, changedBy,inactive_APPusage, statusChangedVia,
batteryDepositedTo, batteryDepositedToR, p.partnerType, finalDskStatus,
t.ticketDate, COALESCE(t.ticketStatus, 'noticket') ticketStatus, t.name,d.noOfBatteries driverBattery
from p1
LEFT JOIN
(select
date(date_add(t.createdAt, interval 330 minute)) ticketDate,
CASE WHEN t.ticketTypeId IS NOT NULL THEN 'ticket' ELSE 'noTicket' END as ticketStatus,
tst.name, t.complainantId
from tickets t LEFT JOIN ticketSubTypes tst ON t.ticketSubTypeId=tst.id
where t.deletedAt IS NULL AND t.ticketTypeId IN (56,103,127)
) t ON p1.driverId=t.complainantId AND p1.inactiveDate=t.ticketDate
LEFT JOIN partners p ON p1.batteryDepositedToR=p.id
left join drivers d on d.id = p1.driverId
order by 1 desc) as ab group by 1,2,3""")

dfInActivity

In [ ]:
dfActivity = prodFetch(f"""SELECT ab.activeDate date, driverId, ab.zoneId, count(*) activity
FROM
(
with activityData as(
select
date(date_add(createdAt, interval 330 minute)) activeDate,
driverId,
changedBy
from driverAudits
where deletedAt IS NULL
AND oldStatus IN ('onLeave', 'toBeActivated')
AND newStatus='active'
AND date(date_add(createdAt, interval 330 minute)) = '{yesterday_date}'
),
p1 as (
select
a.*,
dbas.partnerId,
CASE WHEN isActiveDsk=1 THEN 'DSK' ELSE 'NON_DSK' END dsk_Status,
isActiveDsk,
dskId, dskCreatedAt, dskUpdatedAt,
CASE
WHEN isActiveDsk=1 THEN 'DSK'
WHEN isActiveDsk=0 AND dskId IS NOT NULL AND dsk.status='inactive' AND
(activeDate BETWEEN dskCreatedAt AND dskUpdatedAt) THEN 'DSK'
WHEN isActiveDsk=0 AND dskId IS NOT NULL AND dskCreatedAt<=activeDate AND dsk.status='active' THEN 'DSK'
ELSE 'NON_DSK'
END finalDskStatus,
dsk.status
from activityData a LEFT JOIN
(select date, partnerId, isActiveDsk from dailyBatteriesAtStations where deletedAt IS NULL) dbas
ON a.changedBy=dbas.partnerId AND a.activeDate=dbas.date
LEFT JOIN
(select
id dskId, partnerId, status, zoneId,
date(date_add(createdAt, interval 330 minute)) dskCreatedAt,
date(date_add(updatedAt, interval 330 minute)) dskUpdatedAt
from dsk where deletedAt IS NULL
-- AND status='active'
) dsk ON a.changedBy=dsk.partnerId
)
select
distinct p1.activeDate, driverId, p.zoneId, changedBy, p.partnerType, finalDskStatus,
t.ticketDate, COALESCE(t.ticketStatus, 'noticket') ticketStatus, t.name,d.noOfBatteries
from p1
LEFT JOIN
(select
date(date_add(t.createdAt, interval 330 minute)) ticketDate,
CASE WHEN t.ticketTypeId IS NOT NULL THEN 'ticket' ELSE 'noTicket' END as ticketStatus,
tst.name, t.complainantId
from tickets t LEFT JOIN ticketSubTypes tst ON t.ticketSubTypeId=tst.id
where t.deletedAt IS NULL AND t.ticketTypeId IN (56,103,127)
) t ON p1.driverId=t.complainantId AND p1.activeDate=t.ticketDate
LEFT JOIN partners p ON p1.changedBy=p.id
left join drivers d on d.id = p1.driverId
order by 1 desc
)
as ab
Group by 1,2,3""")

dfActivity

In [ ]:
final_merged = final_merged.merge(dfInActivity, on=['driverId', 'date', 'zoneId'], how='left')

final_merged.fillna(0, inplace=True)
final_merged

In [ ]:
final_merged = final_merged.merge(dfActivity, on=['driverId', 'date', 'zoneId'], how='left')
final_merged.fillna(0, inplace=True)
final_merged

In [ ]:
crossSwap = read("https://docs.google.com/spreadsheets/d/1VhLAxK4WGVPG0KsDsdh-CxMAmdOrdiVsExgNb__TnRQ/edit?gid=0#gid=0", "Sheet1")

crossSwap

In [ ]:
crossSwap = crossSwap.iloc[1:].copy() 
crossSwap.rename(columns={crossSwap.columns[0]: 'zoneId', crossSwap.columns[1]: 'crossSwap %'}, inplace=True)

crossSwap

In [ ]:
ticket_cols = ['battery_totaltickets', 'wait_totaltickets', 'meter_totaltickets', 'crossSwap_totaltickets']
final_merged[ticket_cols + ['inactivity', 'activity']] = final_merged[ticket_cols + ['inactivity', 'activity']].apply(pd.to_numeric, errors='coerce').fillna(0)

zone_summary = final_merged.groupby('zoneId').agg(
    active_drivers=('status', lambda x: (x == 'active').sum()),
    total_drivers=('driverId', 'count'),
    ops_count=('opsFlag', 'sum'),
    revenue_sum=('revenue', 'sum'),
    swaps_sum=('swaps', 'sum'),
    battery_tickets=('battery_totaltickets', 'sum'),
    wait_tickets=('wait_totaltickets', 'sum'),
    meter_tickets=('meter_totaltickets', 'sum'),
    crossswap_tickets=('crossSwap_totaltickets', 'sum'),
    total_inactivity=('inactivity', 'sum'),
    total_activity=('activity', 'sum')
).reset_index()

zone_summary['ops_percent'] = zone_summary['ops_count'] / zone_summary['active_drivers']
zone_summary['revenue_per_swap'] = zone_summary['revenue_sum'] / zone_summary['swaps_sum']
zone_summary['swap_per_ops'] = zone_summary['swaps_sum'] / zone_summary['ops_count']

zone_summary.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

zone_summary

In [ ]:
zone_summary_lastweek = finalLastWeek.groupby('zoneId').agg(
    ops_count_lastweek=('opsFlag', 'sum'),
    total_drivers_lastweek=('driverId', 'count'),
    active_drivers_lastweek=('status', lambda x: (x == 'active').sum())
).reset_index()

zone_summary_lastweek['ops_percent_lastweek'] = zone_summary_lastweek['ops_count_lastweek'] / zone_summary_lastweek['active_drivers_lastweek']

zone_summary_lastweek

In [ ]:
final_df = zone_summary.merge(zone_summary_lastweek[['zoneId', 'ops_percent_lastweek']], on='zoneId', how='left')

final_df['ops_percent_variance_vs_last_week'] = final_df['ops_percent'] - final_df['ops_percent_lastweek']

final_df

In [ ]:
final_df = final_df.merge(rt_final, on='zoneId', how='left')

final_df

In [ ]:
final_df.fillna(0, inplace=True)
final_df

In [ ]:
final_output = final_df[[
    'zoneId',
    'active_drivers',
    'ops_percent',
    'ops_percent_variance_vs_last_week',
    'revenue_per_swap',
    'swap_per_ops',
    'battery_tickets',
    'total_activity',
    'total_inactivity',
    'crossswap_tickets',
    'wait_tickets',
    'meter_tickets',
    'rt_cases'
]].copy()

final_output = final_output.rename(columns={
    'zoneId': 'Zone Id',
    'active_drivers': 'Active Drivers',
    'ops_percent': 'Ops %',
    'ops_percent_variance_vs_last_week': 'Ops% variance same day last week',
    'revenue_per_swap': 'Revenue/Swap',
    'swap_per_ops': 'Swap/Ops',
    'battery_tickets': 'DSK Battery Unavailability Tickets',
    'total_activity': 'Activity',
    'total_inactivity': 'Inactivity',
    'crossswap_tickets': 'Cross Swap Related Tickets',
    'wait_tickets': 'Wait Time Tickets',
    'meter_tickets': 'Meter Issues Tickets',
    'rt_cases': 'RT Cases'
})

final_output = final_output.sort_values(by='Zone Id')

In [ ]:
dfFinal = final_output.merge(crossSwap, left_on = 'Zone Id', right_on='zoneId', how='left').drop(columns = 'zoneId')

dfFinal

In [ ]:
dfFinal.fillna('', inplace=True)
dfFinal

In [ ]:
dfFinal['Active Drivers'] = pd.to_numeric(dfFinal['Active Drivers'], errors='coerce')
dfFinal.sort_values('Active Drivers', ascending=False, inplace=True)
dfFinal.reset_index(drop=True, inplace=True)
dfFinal

In [ ]:
print(dfFinal.columns)

In [ ]:
int_cols = [
    'Active Drivers', 'DSK Battery Unavailability Tickets', 'Activity',
    'Inactivity', 'Cross Swap Related Tickets', 'Wait Time Tickets',
    'Meter Issues Tickets', 'RT Cases'
]

float_cols = ['Revenue/Swap', 'Swap/Ops']
percentage_cols = ['Ops %', 'Ops% variance same day last week', 'crossSwap %']
string_cols = ['Zone Id']

for col in int_cols:
    dfFinal[col] = dfFinal[col].fillna(0).astype(int)

for col in float_cols:
    dfFinal[col] = pd.to_numeric(dfFinal[col], errors='coerce').round(2)

def format_percent(val):
    try:
        if pd.isnull(val) or val == '':
            return ''
        val = val*100
        val = str(val).replace('%', '')
        return f"{float(val):.2f}%"
    except:
        return ''

for col in percentage_cols:
    dfFinal[col] = dfFinal[col].map(format_percent)

for col in string_cols:
    dfFinal[col] = dfFinal[col].astype(str)


In [ ]:
dfFinal

In [ ]:
dfFinal['Ops %'] = pd.to_numeric(dfFinal['Ops %'].str.rstrip('%'), errors='coerce')
dfFinalest = dfFinal[dfFinal['Ops %'] != 0].copy()
dfFinalest['Ops %'] = dfFinalest['Ops %'].map(lambda x: f"{float(x):.2f}%")
dfFinalest

In [ ]:
write("https://docs.google.com/spreadsheets/d/1-irsBnNdhl-tMSxF14rOEKiEAFtM3ebhCNhqFStsdIk/edit?gid=0#gid=0", "Zonal-info", dfFinalest)

In [ ]:
dfLogs = dfFinalest.copy()
dfLogs['date'] = pd.Timestamp.today().date()

dfLogs

In [ ]:
append("https://docs.google.com/spreadsheets/d/1-irsBnNdhl-tMSxF14rOEKiEAFtM3ebhCNhqFStsdIk/edit?gid=0#gid=0", "logs", dfLogs)

# slack bot code

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets.readonly']
SHEET_ID = 'XXX'
LOGGER_CHANNEL_ID = 'XXX'
TARGET_CHANNEL_ID = 'XXX'
POC_ID = 'XXX'
# BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN", 'XXX') # new token
BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN", 'XXX') # old token

def get_access_token(creds):
    creds.refresh(Request())
    return creds.token


def export_sheet_as_jpeg(sheet_id, page_id, file_name):
    return_bool = False

    variables_keys = {
        "type": os.getenv("TYPE"),
        "project_id": os.getenv("PROJECT_ID"),
        "private_key_id": os.getenv("PRIVATE_KEY_ID"),
        "private_key": os.getenv("PRIVATE_KEY"),
        "client_email": os.getenv("CLIENT_EMAIL"),
        "client_id": os.getenv("CLIENT_ID"),
        "auth_uri": os.getenv("AUTH_URI"),
        "token_uri": os.getenv("TOKEN_URI"),
        "auth_provider_x509_cert_url": os.getenv("AUTH_PROVIDER_X509_CERT_URL"),
        "client_x509_cert_url": os.getenv("CLIENT_X509_CERT_URL")
    }

    creds = service_account.Credentials.from_service_account_info(
        info=variables_keys, scopes=SCOPES)

    access_token = get_access_token(creds)
    title = 'dailyZonewiseInfo'
    export_url = (
    f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?'
    f'format=pdf&'
    f'size=7&'
    f'portrait=false&'
    f'sheetnames=false&'
    f'gridlines=false&'
    f'printtitle=false&'
    f'title={title}&'
    f'scale=4&'
    f'fitw=true&'
    f'horizontal_alignment=CENTER&'
    f'gid={page_id}'
)

    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(export_url, headers=headers)

    if response.status_code == 200:
        pdf_bytes = response.content
        pdf_document = fitz.open(stream=pdf_bytes, filetype="pdf")

        zoom = 2
        mat = fitz.Matrix(zoom, zoom)

        for page_number in range(pdf_document.page_count):
            page = pdf_document.load_page(page_number)
            pixmap = page.get_pixmap(matrix=mat)
            png_bytes = pixmap.tobytes('png')
            return_bool = send_image_to_slack(BytesIO(png_bytes), file_name)

    return return_bool


def send_image_to_slack(image_bytes, file_name):
    client = WebClient(token=BOT_TOKEN)

    try:
        response = client.files_upload_v2(
            channel=TARGET_CHANNEL_ID,
            file=image_bytes,
            title=f"{file_name}.jpeg",
            initial_comment=f"{file_name} for {str(dt.now())[:10]}"
        )
        logging.info(f"File {file_name}.jpeg successfully uploaded to Slack.")
        return True

    except SlackApiError as e:
        logging.error(f"Error sending file to Slack: {e.response['error']}")
        return False
    except Exception as e:
        logging.error(f"Unexpected error: {str(e)}")
        return False


def check_conditions():
    return True


def send_slack_message(channel, message):
    client = WebClient(token=BOT_TOKEN)
    try:
        client.chat_postMessage(channel=channel, text=message)
        logging.info("Slack message sent successfully.")
    except SlackApiError as e:
        logging.error(f"Error sending message to Slack: {e.response['error']}")


def main():
    if not check_conditions():
        logging.error("Conditions not met for generating reports.")
        send_slack_message(LOGGER_CHANNEL_ID, f"Daily MIS - Data Error \n<@{POC_ID}>")
        return

    mis_flag = export_sheet_as_jpeg(SHEET_ID, '0', 'Zonal-info')

    if mis_flag:
        logging.info("Reports successfully uploaded to Slack.")
        send_slack_message(LOGGER_CHANNEL_ID, f"Zonewise info Posted for {str(dt.now())[:10]}")
    else:
        logging.error("Error uploading files to Slack.")
        send_slack_message(LOGGER_CHANNEL_ID, f"Error Uploading File To Channel. \n<@{POC_ID}>")


if __name__ == "__main__":
    main()